In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import cv2
import torch
import pickle
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import ParameterGrid

In [6]:
!pwd

/working/kaggle_petfinder/protos


In [2]:
!ls ../exps/exp21_*

convnext_large_4.pth  hist.log	train_cv_score.csv


In [13]:
import torch
import sys
sys.path.append(str('/working/kaggle_petfinder/input/timm-pytorch-image-models/pytorch-image-models-master'))
import timm
sys.path.append('../input/ConvNeXt')
import models.convnext
import models.convnext_isotropic


model = timm.create_model('convnext_large')

model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=1, bias=True)

checkpoint1 = torch.load(f'../exps/exp21_convnext_large/convnext_large_4.pth')
model.load_state_dict(checkpoint1)

<All keys matched successfully>

In [21]:
import os
from PIL import Image
import imagehash
from tqdm.auto import tqdm
if 0:
    df1 = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
    df1['path'] = '../input/petfinder-adoption-prediction/train_images/' + df1['PetID'] + '-1.jpg'
    df1 = df1[df1['path'].map(os.path.exists)]

    df2 = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
    df2['path'] = df2['Id'].map(lambda x:str(f'../input/petfinder-pawpularity-score/train/{x}.jpg'))

    df1['hash'] = [imagehash.average_hash(Image.open(x))
                        for x in tqdm(df1['path'].values)]
    df2['hash'] = [imagehash.average_hash(Image.open(x))
                        for x in tqdm(df2['path'].values)]
    #train_df_prev['hash'] = [imagehash.average_hash(Image.open(x))
    #                    for x in tqdm(train_df_prev['path'].values)]

    df1.to_csv('df1.csv', index=False)
    df2.to_csv('df2.csv', index=False)
else:
    df1 = pd.read_feather('../input/pet-data/df_allfeats.ftr')
    #df1['AdoptionSpeed'] += pd.read_csv('../input/pet-data/df1_train_test_v3.csv')['AdoptionSpeed']
    #df1['AdoptionSpeed'] /= 2
    df2 = pd.read_csv('../input/pet-data/df2.csv')

In [6]:
df1.T

,0,1,2,3,4,5,6,7,8,9,...,71825,71826,71827,71828,71829,71830,71831,71832,71833,71834
PetID,d336e6953,77544cbd4,0149d04e7,cad25c91b,9c7476c25,e01f077a7,449290a8b,6c9382d17,ebdf19959,6a864757b,...,af7305535,f51767bbe,c46f3bb03,ef20ce351,8a69b8ca9,819610fb0,280dd2698,4b393cb28,b88cb23c9,311368756
hash,193f20c0c86c2f2f,0f0f0f3fffffe000,387fef070743071f,0c00003f27e7ffff,ffb301d9f0e08e04,1f1f40602c1e9f9f,ffc7c3c7c3c3d3a7,fff860000083feff,18981c371fff0707,f0fcece060302040,...,0078784cecfdffff,78787c1c18797b38,0261f03cbaf1e87c,0f0f79f1f0f8f8b8,00000c1d9f8fffff,00003e3f3f1f0000,00001c1e6e7f767c,3070380cfbffffff,f000faffe7c3c111,78cf3c0d08e0e0fd
inception_resnet_87,0.414027,0.208592,0.184807,0.251402,0.376016,0.504619,0.248497,0.371604,0.399966,0.323036,...,0.55442,0.329664,0.405163,0.426109,0.254018,0.222457,0.317543,0.484931,0.284259,0.304412
gnvec172,-0.030165,-0.055317,-0.052831,-0.041911,-0.039633,-0.001962,-0.005127,-0.010763,-0.031649,-0.05068,...,-0.039659,-0.063658,-0.051838,-0.038266,-0.070636,-0.041696,-0.051321,-0.02378,-0.039963,-0.0604
dense121_2_75,0.158258,0.182523,0.105973,0.122891,0.142386,0.311862,0.166855,0.157866,0.284791,0.292992,...,0.125362,0.173067,0.104174,0.290988,0.174089,0.184677,0.131595,0.141036,0.097768,0.228855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
meta_label_score_min_MAX,0.598356,0.746334,0.670161,0.599829,0.712055,0.699902,0.683355,0.562045,0.72713,0.699434,...,0.604277,0.688773,0.55469,0.640164,0.704174,0.769948,0.777979,0.772419,0.670216,0.705534
SVD_CHAR_Name_9,-0.00973,-0.058482,-0.039846,-0.041887,0.110048,0.011499,-0.148574,-0.041405,0.004895,-0.12464,...,0.008636,-0.00073,-0.044329,0.0,0.065091,0.073473,0.008316,0.0,-0.038658,-0.073724
SVD_Description_67,0.000989,-0.00771,0.053205,-0.004458,0.132795,-0.00743,-0.030556,0.024565,-0.001819,-0.000635,...,-0.000789,-0.00624,-0.019076,-0.005478,-0.01024,-0.012016,0.006941,0.011055,0.03802,0.00299
SVD_CHAR_Name_14,-0.007476,-0.000942,0.035218,-0.005269,-0.033357,-0.001937,-0.109012,0.009306,-0.029861,0.137073,...,-0.004133,-0.016302,-0.021762,0.0,0.055032,0.045667,-0.004573,0.0,-0.132645,0.087472


In [7]:
if 0:
    from sklearn.feature_extraction.text import TfidfVectorizer
    model = TfidfVectorizer(stop_words=None,#np.load('stop_words.npy').tolist(),
                                binary=False,
                                norm='l2',
                                #analyzer='char_wb',
                                #ngram_range=(2, 5),
                                # tokenizer=tokenizer,
                                # max_features=50000,
                                #max_df=50,
                                min_df=2,
                                dtype=np.float32)

    #data = model.fit_transform(df1['Description'].fillna('').str.lower().values)
    #df1['desc_sum'] = np.array(data.sum(axis=1))[:, 0]
    #df1['desc_max'] = np.array(data.max(axis=1).todense())[:, 0]
    #df1['desc_mean'] = df1['desc_sum'] / (data.getnnz(axis=1))


    data = model.fit_transform(df1['Name'].fillna('').str.lower().values)
    print(data.shape)
    df1['name_sum'] = np.array(data.sum(axis=1))[:, 0]
    df1['name_max'] = np.array(data.max(axis=1).todense())[:, 0]
    df1['name_mean'] =  df1['name_sum'] / data.getnnz(axis=1)

In [8]:
if 0:
    import imagesize
    list_label = []
    for path in tqdm(df2['path'].values):
        list_label.append(imagesize.get(path))

    #df['resnet_label'] = list_label
    list_label = np.array(list_label)
    df2['img_h'] = list_label[:, 0]
    df2['img_w'] = list_label[:, 1]

    df2['aspect_ratio'] = df2['img_h'] / df2['img_w']

In [9]:
df1.shape, df1.hash.nunique(), df2.hash.isin(df1.hash).sum()

((71835, 203), 70437, 1942)

In [10]:
df2 = pd.merge(df2, pd.read_csv('objdet.csv'), how='left', on='path')

In [16]:
#df1.to_csv('train_last_comp.csv', index=False)
df = pd.merge(df2, df1.drop('PetID', axis=1), how='left', on='hash')

In [17]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')['Id'].map(lambda x:str(f'../input/petfinder-pawpularity-score/train/{x}.jpg'))
oof = pd.concat([train,
                 pd.read_csv('../input/petfinder-ensumble/exp085_ensumble/exp085_ensumble_oof.csv')['oof']], 
                 axis=1).set_index('Id')['oof']/100
oof2 = pd.concat([train,
                 pd.read_csv('../input/exp108/exp108_oof.csv')['pred']], 
                 axis=1).set_index('Id')['pred']/100


#df['pred'] = df['path_x'].map(pd.read_csv('../input/pet-data/ens_exp15.csv', index_col='path')['pred'])
df['pred'] = df['path'].map(pd.read_csv('../input/pet-data/ens_exp9.csv', index_col='path')['pred'])
#df['pred'] = df['path_x'].map(pd.read_csv('../input/pet-data/ens_exp9.csv', index_col='path')['pred'])
df['pred'] += df['path'].map(oof)
df['pred'] += df['path'].map(oof2)
df['pred'] /= 3
#df['pred'] = df[['pred1', 'pred2', 'pred3', 'pred4']].values.mean(axis=1)
#df['pred']
#df[['pred1', 'pred2', 'pred3', 'pred4']]

df.to_feather('df_allfeat_merged.ftr')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  # This is added back by InteractiveShellApp.init_path()


In [22]:
df2.shape

(9912, 16)

In [23]:
!wc -l ../input/petfinder-pawpularity-score/train.csv

9913 ../input/petfinder-pawpularity-score/train.csv


In [11]:
df['len_desc'] = df['Description'].fillna('').map(len)
df['len_word_desc'] = df['Description'].fillna('').map(lambda x: len(x.split()))
for n in ['No Name Yet']:#, 'No Name', 'Unknown']:
    df['Name'] = df['Name'].fillna('').replace(n, '')
df['len_name'] = df['Name'].map(len)
df['len_word_name'] = df['Name'].fillna('').map(lambda x: len(x.split()))

df['cnt_RescuerID'] = df['RescuerID'].map(df1['RescuerID'].value_counts())

In [12]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,10008,10009,10010,10011,10012,10013,10014,10015,10016,10017
Id,0007de18844b0dbbb5e1f607da0606e0,0009c66b9439883ba2750fb825e1d7db,0013fd999caf9a3efe1352ca1b0d937e,0018df346ac9c1d8413cfcc888ca8246,001dc955e10590d3ca4673f034feeef2,001dd4f6fafb890610b1635f967ea081,0023b8a3abc93c712edd6120867deb53,0031d6a9ef7340f898c3e05f92c7bb04,0042bc5bada6d1cf8951f8f9f0d399fa,0049cb81313c94fa007286e9039af910,...,ff78a56d5e0029cd6288c62f7b7f10e5,ff903858a9899465ee4cd17e942bef24,ff9b3b3468c4b35a3b6450a1ddc88449,ffaf7bf7e64b9eecaf02fc2c14d6b654,ffb8b466494a3e48bcb874a2fe9b976d,ffbfa0383c34dc513c95560d6e1fdb57,ffcc8532d76436fc79e50eb2e5238e45,ffdf2e8673a1da6fb80342fa3b119a20,fff19e2ce11718548fa1c5d039a5192a,fff8e47c766799c9e12f3cb3d66ad228
Subject Focus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Eyes,1,1,1,1,0,0,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
Face,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
Near,1,0,1,1,1,0,1,0,1,1,...,1,0,1,1,1,1,1,1,1,1
Action,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Accessory,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Group,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Collage,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Human,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0


In [13]:
df2.columns

Index(['Id', 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'Pawpularity',
       'path', 'hash', 'img_h', 'img_w', 'aspect_ratio', 'det_label',
       'det_left', 'det_top', 'det_right', 'det_bottom', 'det_prob'],
      dtype='object')

In [14]:
df.columns

Index(['Id', 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'Pawpularity',
       'path_x', 'hash', 'img_h', 'img_w', 'aspect_ratio', 'det_label',
       'det_left', 'det_top', 'det_right', 'det_bottom', 'det_prob', 'path_y',
       'is_train', 'is_test', 'idx', 'PetID', 'Type', 'Name', 'Age', 'Breed1',
       'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health',
       'Quantity', 'Fee', 'State', 'RescuerID', 'VideoAmt', 'Description',
       'PhotoAmt', 'AdoptionSpeed', 'len_desc', 'len_word_desc', 'len_name',
       'len_word_name', 'cnt_RescuerID'],
      dtype='object')

In [15]:
#df = pd.read_csv('../exps/exp1_swin_large_patch4_window12_384/train_cv_score.csv')
#df = pd.read_csv('ens_exp15.csv')

In [16]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')['Id'].map(lambda x:str(f'../input/petfinder-pawpularity-score/train/{x}.jpg'))
oof = pd.concat([train,
                 pd.read_csv('../input/petfinder-ensumble/exp085_ensumble/exp085_ensumble_oof.csv')['oof']], 
                 axis=1).set_index('Id')['oof']/100
oof2 = pd.concat([train,
                 pd.read_csv('../input/exp108/exp108_oof.csv')['pred']], 
                 axis=1).set_index('Id')['pred']/100


#df['pred'] = df['path_x'].map(pd.read_csv('../input/pet-data/ens_exp15.csv', index_col='path')['pred'])
df['pred'] = df['path_x'].map(pd.read_csv('../input/pet-data/ens_exp9.csv', index_col='path')['pred'])
#df['pred'] = df['path_x'].map(pd.read_csv('../input/pet-data/ens_exp9.csv', index_col='path')['pred'])
df['pred'] += df['path_x'].map(oof)
df['pred'] += df['path_x'].map(oof2)
df['pred'] /= 3
#df['pred'] = df[['pred1', 'pred2', 'pred3', 'pred4']].values.mean(axis=1)
#df['pred']
#df[['pred1', 'pred2', 'pred3', 'pred4']]

In [17]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

df['fold'] = -1


N_FOLDS = 5
strat_kfold = StratifiedKFold(n_splits=5, random_state=365, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(df.index, df['Pawpularity'])):
    df.loc[train_index, 'fold'] = i

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [29]:
COL_FEATURES = ['Blur', 
                #'Face', 
                #'Occlusion',
                'Eyes',
                #'Accessory',
               #'Group',
                #'Human',
                #'Near',
                #'Collage',
                #'Info',# 
                #'Action',
                
                'pred',
                #'pred1', 'pred2', 'pred3', 'pred4',
                
                #'len_desc', 'len_name', 
                #'aspect_ratio',
                #'len_word_desc', 'len_word_name',
                #'isin'
                #'breed_label', 'breed_prob',
                #'img_h', 'img_w',
                #'det_label', 
                #'det_left', 'det_top', 'det_right', 'det_bottom',
                #'det_prob'
               ] + df1.drop(['hash', 'PetID'
                            ], axis=1).columns.tolist()

#COL_FEATURES = COL_FEATURES[:7]

In [28]:
aaa = [1, 2, 3]
aaa.pop(0)
aaa

[2, 3]

In [30]:
np.save('opt_final', np.array(COL_FEATURES, dtype=str))

In [19]:
def rmse(preds, train_data):
    labels = train_data.get_label()
    #import pdb;pdb.set_trace()
    loss = np.sqrt(((labels - preds.clip(0.01, 1)) ** 2).mean()) * 100
    
    #loss = np.sqrt(((labels - preds.reshape(-1, 100).argmax(axis=1)) ** 2).mean())
    return 'rmse', loss, False


def train(fold, param):
     
    X_train = df.loc[df['fold'] != fold, COL_FEATURES]
    y_train = df.loc[df['fold'] != fold, 'Pawpularity'].values  / 100
    
    X_valid = df.loc[df['fold'] == fold, COL_FEATURES]
    y_valid = df.loc[df['fold'] == fold, 'Pawpularity'].values  / 100
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
    
    model = lgb.train(param,
                      train_data,
                      valid_sets=valid_data,
                      #early_stopping_rounds=50,
                      verbose_eval=100,
                      feval=rmse
                      )
    return model

def train_all(param):
     
    X_train = df.loc[:, COL_FEATURES]
    y_train = df.loc[:, 'Pawpularity'].values  / 100
    
    X_valid = df.loc[:, COL_FEATURES]
    y_valid = df.loc[:, 'Pawpularity'].values  / 100
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
    
    model = lgb.train(param,
                      train_data,
                      valid_sets=valid_data,
                      early_stopping_rounds=50,
                      verbose_eval=100,
                      feval=rmse
                      )
    return model

In [20]:
if 1: 
    all_params = {'objective': ['tweedie', 'mse', 'xentropy'],
                  'tweedie_variance_power': [1.2],
                 'verbosity': [-1],
                 'boosting_type': ['gbdt'],
                 'feature_pre_filter': [False],
                 #'bagging_fraction': [0.7, 0.8, 0.9, 1, 0.95],
                 'bagging_freq': [1],
                 'num_iterations': [10000],
                 'early_stopping_round': [100],
                 'n_jobs': [16],
                 'seed': [114],
                 'metric':  ['None'],  # trial.suggest_categorical('metric', ['auc', 'binary_logloss', ]), #'auc',
                 'learning_rate': [0.05],
                  'lambda_l1': [0],
                  'lambda_l2': [1],
                  'min_child_samples': [150, 140, 160],
                  'num_leaves': [7, 15, 31],
                  'feature_fraction': [0.8, 0.9, 0.7],
                  'min_gain_to_split': [0.02, 0.01],
                  'linear_tree': [False],
                  #'max_bins': [8, 16, 32, 62, 128, 256, 512]
                 }
else:
    all_params = {'objective': ['tweedie'],
                  'tweedie_variance_power': [1.2],
                 'verbosity': [-1],
                 'boosting_type': ['gbdt'],
                 'feature_pre_filter': [False],
                 #'bagging_fraction': [0.7, 0.8, 0.9, 1, 0.95],
                 'bagging_freq': [1],
                 'num_iterations': [10000],
                 'early_stopping_round': [100],
                 'n_jobs': [16],
                 'seed': [114],
                 'metric':  ['None'],  # trial.suggest_categorical('metric', ['auc', 'binary_logloss', ]), #'auc',
                 'learning_rate': [0.05],
                  'lambda_l1': [0],
                  'lambda_l2': [1],
                  'min_child_samples': [150],
                  'num_leaves': [7],
                  'feature_fraction': [0.8],
                  'min_gain_to_split': [0],
                  'linear_tree': [False],
                  #'max_bins': [8, 16, 32, 62, 128, 256, 512]
                 }

In [21]:
best_score = 1.0e10
best_param = None
for param in tqdm(ParameterGrid(all_params)):
    print(param)
    list_loss = []
    list_imp = []
    list_num = []
    for fold in range(5):
        model = train(fold, param)
        sc = model.best_score['valid_0']['rmse']# * 100
        list_loss.append(sc)
        list_num.append(model.best_iteration)
        
        imp = pd.DataFrame(model.feature_importance(importance_type='gain'), columns=['imp'])
        imp['col'] = COL_FEATURES
        list_imp.append(imp.set_index('col'))
    sc = np.mean(list_loss)
    if sc < best_score:
        best_score = sc
        best_param = param

  0%|          | 0/162 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


{'bagging_freq': 1, 'boosting_type': 'gbdt', 'early_stopping_round': 100, 'feature_fraction': 0.8, 'feature_pre_filter': False, 'lambda_l1': 0, 'lambda_l2': 1, 'learning_rate': 0.05, 'linear_tree': False, 'metric': 'None', 'min_child_samples': 150, 'min_gain_to_split': 0.02, 'n_jobs': 16, 'num_iterations': 10000, 'num_leaves': 7, 'objective': 'tweedie', 'seed': 114, 'tweedie_variance_power': 1.2, 'verbosity': -1}
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 17.2706
[200]	valid_0's rmse: 17.2963
Early stopping, best iteration is:
[109]	valid_0's rmse: 17.2661
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 16.7632
[200]	valid_0's rmse: 16.7321
Early stopping, best iteration is:
[148]	valid_0's rmse: 16.7231
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 17.0516
Early stopping, best iteration is:
[83]	valid_0's rmse: 17.0402
Training until validation scores don't improve for 10

In [22]:
17.122953793907463
17.12237161917507
17.12206798235878
17.035855591812624
17.02063665426032
17.013295893520258
17.012832654542915
17.007733147459998
17.00092348180123
16.997493135137915

17.00197311363724
17.040035089681957
17.006794943635573

print(best_param)
print(best_score)

{'bagging_freq': 1, 'boosting_type': 'gbdt', 'early_stopping_round': 100, 'feature_fraction': 0.9, 'feature_pre_filter': False, 'lambda_l1': 0, 'lambda_l2': 1, 'learning_rate': 0.05, 'linear_tree': False, 'metric': 'None', 'min_child_samples': 160, 'min_gain_to_split': 0.02, 'n_jobs': 16, 'num_iterations': 10000, 'num_leaves': 7, 'objective': 'tweedie', 'seed': 114, 'tweedie_variance_power': 1.2, 'verbosity': -1}
17.013396234718062


In [23]:
best_param['num_iterations'] = round(np.mean(list_num) * 1.1)

In [24]:
model = train_all(best_param)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 16.7859
Did not meet early stopping. Best iteration is:
[105]	valid_0's rmse: 16.7731


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


In [25]:
aaa = 0
for i in range(5):
    aaa += np.sqrt(((df.loc[df['fold']==i, 'pred'] * 100 - df.loc[df['fold']==i, 'Pawpularity']) ** 2).mean())
aaa / 5

17.114673475058627

In [26]:
with open('model_lgb_last_comp_all.pkl', 'wb') as f:
    pickle.dump(model, f, -1)

In [27]:
#imp = pd.DataFrame(model.feature_importance(importance_type='gain'), columns=['imp'])
#imp['col'] = COL_FEATURES
imp = sum(list_imp) / 5
imp.sort_values('imp', ascending=False)

,imp
col,
pred,4578.714386
Breed1,109.926845
aspect_ratio,98.596628
img_h,71.605551
img_w,51.979248
Age,37.846402
PhotoAmt,37.682260
AdoptionSpeed,34.016435
len_name,22.001605


In [28]:
model.feature_name()

['Blur',
 'Eyes',
 'pred',
 'len_desc',
 'len_name',
 'aspect_ratio',
 'len_word_desc',
 'len_word_name',
 'img_h',
 'img_w',
 'is_train',
 'is_test',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'FurLength',
 'Vaccinated',
 'Quantity',
 'Fee',
 'State',
 'PhotoAmt',
 'AdoptionSpeed']